<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Filter OpenTelemetry Spans</h1>

This tutorial shows how to filter OpenTelemetry spans based on a condition.

If you're using multiple OTEL-compatible libraries, you may want to filter spans based on a condition. This can prevent you from sending spans to multiple backends unnecessarily.

There are multiple approaches to do this, in this tutorial we'll show how to do this using a custom SpanProcessor.

### Install Dependencies

In [ ]:
!pip install -q uv
!uv pip install --system -q opentelemetry-sdk opentelemetry-exporter-otlp-proto-http openinference-instrumentation-openai openai python-dotenv 'httpx<0.28'

In [ ]:
import os

from dotenv import load_dotenv
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SpanExporter, SpanProcessor

load_dotenv()

from getpass import getpass

import openai
from openinference.instrumentation.openai import OpenAIInstrumentor

# Define a custom SpanProcessor

We'll extend the SpanProcessor class to add a condition that determines whether a span should be exported.

In [ ]:
class ConditionalSpanProcessor(SpanProcessor):
    def __init__(self, exporter: SpanExporter, condition: callable):
        self.exporter = exporter
        self.condition = condition

    def on_start(self, span, parent_context):
        pass

    def on_end(self, span):
        # Only export spans that meet the condition
        if self.condition(span):
            self.exporter.export([span])

    def shutdown(self):
        self.exporter.shutdown()

    def force_flush(self, timeout_millis=None):
        self.exporter.force_flush(timeout_millis)

Along with this class, we'll define two conditions: one for sending spans to the console, and one for sending spans to Phoenix.

In [ ]:
# Define conditions for sending spans to specific exporters


def console_condition(span):
    return "console" in span.name  # Example: send to Console if "console" is in the span name


def phoenix_condition(span):
    # return "phoenix" in span.name  # Example: send to Phoenix if "phoenix" is in the span name
    return not console_condition(
        span
    )  # Example: send to Phoenix if "console" is not in the span name

# Use our custom SpanProcessor to set up instrumentation
In this example, we'll use Phoenix as one of our destinations, and the console as the other. You could instead add any other exporters you'd like in this approach.

If you need to set up an API key for Phoenix, you can do so [here](https://app.phoenix.arize.com/).

In [ ]:
if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass("Enter your Phoenix API Key: ")

### Define the console exporter

In [ ]:
tracer_provider = TracerProvider()

# Create the Console exporter
console_exporter = ConsoleSpanExporter()

# Add the Console exporter to the tracer provider
tracer_provider.add_span_processor(ConditionalSpanProcessor(console_exporter, console_condition))

### Define the Phoenix exporter

In [ ]:
# Add Phoenix API Key to the headers for tracing and API access
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={phoenix_api_key}"

# Create the Phoenix exporter
otlp_exporter = OTLPSpanExporter(endpoint="https://app.phoenix.arize.com/v1/traces")

# Add the Phoenix exporter to the tracer provider
tracer_provider.add_span_processor(ConditionalSpanProcessor(otlp_exporter, phoenix_condition))

### Add the exporters to the tracer provider

In [ ]:
# Set the tracer provider
trace_api.set_tracer_provider(tracer_provider)

# Run our app and view traces

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

This first span will be exported to the console. Here we're using manual instrumentation to create the span.

In [ ]:
# Create a tracer
tracer = trace_api.get_tracer(__name__)

# Example of creating and exporting spans
with tracer.start_as_current_span("console-span"):
    print("This span will be exported to Console only.")

This next span will be exported only to Phoenix. In this case, we're using our auto-instrumentor for OpenAI to generate the span.

In [ ]:
# Auto-instrumentors can still be used with this setup
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

In [ ]:
client = openai.OpenAI()
client.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role": "user", "content": "Hello, world!"}]
)